In [1]:
import matplotlib.pyplot as plt
#plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['font.family'] ='NanumGothic'
plt.rcParams['axes.unicode_minus'] =False

In [4]:
import os                        
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [10]:
# utils
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import warnings;warnings.filterwarnings('ignore')

# Model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

#import eli5
#from eli5.sklearn import PermutationImportance

In [20]:
trainset_0715= pd.read_csv(r'/data/ephemeral/home/yjl/upstage-ml-regression-ml5/data_final/trainset_0715.csv')

In [22]:
trainset_0715.columns

Index(['시군구', '번지', '본번', '부번', '아파트명', '전용면적(㎡)', '계약년월', '계약일', '층', '건축년도',
       ...
       '대형학원수', '종합학원수', '입시학원수', '외국어학원수', '예체능학원수', '독서실수', '정보학원수',
       '특수교육학원수', '직업기술학원수', '기타학원수'],
      dtype='object', length=116)

In [13]:
new_columns = ['전용면적(㎡)', '구', '동', '계약년월', '층', 
       '건축년도', 'k-건설사(시공사)',
       '국고채-회사채', '서울특별시_가계대출', '서울특별시_주택담보대출',
       '서울특별시_주택매매거래량', '서울특별시_아파트매매거래량', '서울특별시_주택미분양',
       '동수', '세대수', '소형학원수', '중형학원수', '대형학원수', '종합학원수', '입시학원수',
       '외국어학원수', '예체능학원수', '독서실수', '정보학원수', '특수교육학원수', '직업기술학원수', '기타학원수',
       '대장아파트_거리', '계약년',
       'lower_outliers', 'upper_outliers', '모든학원수',
       'Nearest_Bus_Station_Distance', 'Nearest_Subway_Station_Distance','서울특별시_부동산소비심리지수','미분양_구']

In [14]:
train_for_model = trainset_0714[trainset_0714['is_test']==0][new_columns]

In [15]:
test_for_model = trainset_0714[trainset_0714['is_test']==1][new_columns]

In [16]:
# 파생변수 제작으로 추가된 변수들이 존재하기에, 다시한번 연속형과 범주형 칼럼을 분리해주겠습니다.
continuous_columns = []
categorical_columns = []

for column in train_for_model.columns:
    if pd.api.types.is_numeric_dtype(train_for_model[column]):
        continuous_columns.append(column)
    else:
        categorical_columns.append(column)

print("연속형 변수:", continuous_columns)
print("범주형 변수:", categorical_columns)

연속형 변수: ['전용면적(㎡)', '계약년월', '층', '건축년도', '국고채-회사채', '서울특별시_가계대출', '서울특별시_주택담보대출', '서울특별시_주택매매거래량', '서울특별시_아파트매매거래량', '서울특별시_주택미분양', '동수', '세대수', '소형학원수', '중형학원수', '대형학원수', '종합학원수', '입시학원수', '외국어학원수', '예체능학원수', '독서실수', '정보학원수', '특수교육학원수', '직업기술학원수', '기타학원수', '대장아파트_거리', '계약년', 'lower_outliers', 'upper_outliers', '모든학원수', 'Nearest_Bus_Station_Distance', 'Nearest_Subway_Station_Distance', '서울특별시_부동산소비심리지수', '미분양_구']
범주형 변수: ['구', '동', 'k-건설사(시공사)']


In [18]:
# 아래에서 범주형 변수들을 대상으로 레이블인코딩을 진행해 주겠습니다.

# 각 변수에 대한 LabelEncoder를 저장할 딕셔너리
label_encoders = {}

# Implement Label Encoding
for col in tqdm(categorical_columns):
    lbl = LabelEncoder()

    # 모든 데이터를 문자열로 변환하여 Label-Encoding을 fit
    lbl.fit(np.unique(np.concatenate((train_for_model[col], test_for_model[col])).astype(str)))
    train_for_model[col] = lbl.transform(np.array(train_for_model[col]).astype(str))
    test_for_model[col] = lbl.transform(np.array(test_for_model[col]).astype(str))
    label_encoders[col] = lbl           # 나중에 후처리를 위해 레이블인코더를 저장해주겠습니다.

    # Label-Encoding을 fit
    lbl.fit( train_for_model[col].astype(str) )
    train_for_model[col] = lbl.transform(train_for_model[col].astype(str))
    label_encoders[col] = lbl           # 나중에 후처리를 위해 레이블인코더를 저장해주겠습니다.

    # Test 데이터에만 존재하는 새로 출현한 데이터를 신규 클래스로 추가해줍니다.
    for label in np.unique(test_for_model[col]):
      if label not in lbl.classes_: # unseen label 데이터인 경우
        lbl.classes_ = np.append(lbl.classes_, label) # 미처리 시 ValueError발생하니 주의하세요!

    test_for_model[col] = lbl.transform(test_for_model[col].astype(str))

 67%|██████▋   | 2/3 [00:04<00:02,  2.44s/it]


ValueError: y contains previously unseen labels: '1133'